Notebook to test models in `cdensity.py` and CondSource in `cdata.py`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'



In [ ]:
import torch
import torch.distributions as dists

import kcgof
import kcgof.log as klog
import kcgof.util as util
import kcgof.cdensity as cden
import kcgof.cdata as cdat
import kcgof.plot as plot

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 24
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Ordinary least squares with Gaussian noise

In [ ]:
# 2d
dx = 1
n = 50
X = torch.randn(n, dx)
slope = torch.tensor([2.0])

cs = cdat.CSGaussianOLS(slope=slope, c=-1.0, variance=1.0)


In [ ]:
Y = cs(X, seed=1)

In [ ]:
plt.plot(X,Y, 'bo')

## Least squares with heteroscedastic Gaussian noise

In [ ]:
# 2d
dx = 1
n = 100
rx = dists.Normal(0, 1)
X = rx.sample((n, dx))

# model
def f_mean(X):
    return torch.mean(X, dim=1, keepdim=True)

def f_variance(X):
#     \sigma^2(x) = 0.5 + exp(-(x - 1)^2/0.1^2).
    std = 0.2
    spike_var = 5.0
    return 0.5 + spike_var*torch.exp(-(X-1.0)**2/std**2)
    
p = cden.CDGaussianHetero(f_mean, f_variance, dx=dx )
cs = p.get_condsource()
# cs = cdat.CSGaussianOLS(slope=slope, c=-1.0, variance=1.0)

# sample Y from the cond source
Y = cs(X, seed=3)

In [ ]:
ep = 0.7
# make a grid that covers X

domX = torch.linspace(torch.min(X)-ep, torch.max(X)+ep, 100)
domY = torch.linspace(torch.min(Y).item()-ep, torch.max(Y).item()+ep, 200)


fig, axes = plot.plot_2d_cond_model(
    p, 
    lambda X: torch.exp(rx.log_prob(X)), 
    X, Y, domX=domX, domY=domY, 
    height_ratios=[2,1],
    cmap='pink_r', levels=50)
# plt.xlabel('$x$')
# plot.plot_2d_cond_data(X, Y)